# 키즈카페 예약 정보 크롤링

## 1. 라이브러리 불러오기
크롤링 및 데이터 처리에 필요한 주요 라이브러리를 불러옵니다.

In [11]:
import time
from bs4 import BeautifulSoup
import pandas as pd
import re
import datetime
import urllib
import requests
import json
import numpy as np
from datetime import datetime
import os

## 2. 날짜 및 저장시간 설정
크롤링할 날짜와 데이터 저장용 타임스탬프를 지정합니다.

In [12]:
month = '05'                     # 예약 날짜의 월
want_day = [19]                      # 예약 날짜 리스트
crawling_dt = datetime.now().strftime('%Y-%m-%d-%H')

## 3. 대상 키즈카페 목록 불러오기
CSV 파일로부터 예약 대상 키즈카페 리스트를 불러옵니다.

In [13]:
df = pd.read_csv('kids_cafe_list.csv')
kids_cafe_urls = df['예약 신청 URL'].loc[:]
kids_cafe_data = []

## 4. 예약 현황 크롤링 및 데이터 수집
각 키즈카페의 예약 페이지에 접속하여 원하는 날짜의 예약 현황을 추출합니다.

In [14]:
for url in kids_cafe_urls:
    if not url:
        continue

    response = requests.get(url)
    if response.status_code != 200:
        print(f"URL 로드 실패: {url}")
        continue

    soup = BeautifulSoup(response.content, 'html.parser')
    name = soup.select_one("#container > div > div.write_ty01.m_write_ty01.wr_type > table > tbody > tr:nth-child(1) > td:nth-child(2)").text.strip()
    elements = soup.select("#calendar > tbody > tr")
    data = {"키즈카페 이름": name, "예약 신청 URL": url}

    for element in elements:
        title_td = element.find_all('td')
        for title in title_td:
            try:
                date_number = int(re.findall(r'\d+', title.get('title'))[0])
                if date_number in want_day:
                    for p in title.find_all('p'):
                        session_info = p.get_text(strip=True)
                        session_info = ' '.join(session_info.split())

                        if "프로그램" in session_info:
                            match = re.match(r'(\d+회)프로그램', session_info)
                            if match:
                                session = match.group(1)
                                column_name = f"{month}{date_number}_{session}"
                                data[column_name] = "프로그램"
                        else:
                            match = re.match(r'(\d+회)(개인|단체|공용)(\d+)', session_info)
                            if match:
                                session, user_type, count = match.groups()
                                column_name = f"{month}{date_number}_{session}"
                                data[column_name] = f"{user_type}_{count}"
            except:
                continue
    kids_cafe_data.append(data)

In [15]:
df

,키즈카페 이름,자치구,주소,이용연령_min,이용연령_max,이용정원_개인,이용정원_단체,예약 신청 URL
0,서울형 키즈카페 시립 1호점,동작구,서울특별시 동작구 노량진로 10 서울가족플라자 지하2층 (대방동),4,10,33,33,https://icare.seoul.go.kr/icare/user/kidsCafeR...
1,서울형 키즈카페 시립 뚝섬자벌레점,광진구,서울특별시 광진구 강변북로 2202 2층 꿈틀나루 (자양동),0,6,43,43,https://icare.seoul.go.kr/icare/user/kidsCafeR...
2,서울형 키즈카페 시립 목동점,양천구,서울특별시 양천구 안양천로 1131 지식산업센터 2층 (목동),3,12,24,24,https://icare.seoul.go.kr/icare/user/kidsCafeR...
3,서울형 키즈카페 강남구 역삼1동점,강남구,서울특별시 강남구 언주로107길 4 2층 (역삼동) 2층 서울형 키즈카페,0,6,24,24,https://icare.seoul.go.kr/icare/user/kidsCafeR...
4,서울형 키즈카페 강동구 고덕2동점(아이·맘 강동),강동구,서울특별시 강동구 고덕로 353 일반상가 2층 (농폅건물2층),0,6,12,12,https://icare.seoul.go.kr/icare/user/kidsCafeR...
...,...,...,...,...,...,...,...,...
70,은평아이맘놀이터 수색동점,은평구,"서울특별시 은평구 은평터널로 27 101동 1층 은평아이맘놀이터 3호점 (수색동, ...",0,6,14,14,https://icare.seoul.go.kr/icare/user/kidsCafeR...
71,서울형 키즈카페 종로구 혜화동점(종로 혜명 아이들 상상놀이터),종로구,서울특별시 종로구 성균관로 91 올림픽기념국민생활관 2층 (혜화동),4,8,25,30,https://icare.seoul.go.kr/icare/user/kidsCafeR...
72,서울형 키즈카페 중구 중림동점(노리몽땅),중구,서울특별시 중구 서소문로6길 16 본관 1층 (중림동),0,6,38,38,https://icare.seoul.go.kr/icare/user/kidsCafeR...
73,서울형 키즈카페 중랑구 망우본동점(중랑실내놀이터 양원),중랑구,"서울특별시 중랑구 용마산로 670 시티원스퀘어 상가 201동 지하1층 (망우동, 신...",2,8,73,80,https://icare.seoul.go.kr/icare/user/kidsCafeR...


## 5. 정원 관련 정보 결합
다른 데이터프레임(df2)에서 개인/단체 정원 정보를 불러와 결합합니다.

In [ ]:
df1=pd.read_csv('29_30_re_2025-03-28-20.csv',encoding='cp949') # 크롤링 데이터
df2=pd.read_csv('kids_cafe_list.csv',encoding='UTF-8-SIG') # list 데이터


In [17]:
df1.head()

,키즈카페 이름,예약 신청 URL,0329_1회,0329_2회,0329_3회,0329_4회,0329_5회,0330_1회,0330_2회,0330_3회,0330_4회,0330_5회,crawling_dt
0,서울형 키즈카페 시립 1호점,https://icare.seoul.go.kr/icare/user/kidsCafeR...,개인_0,개인_0,개인_0,개인_0,개인_0,개인_0,개인_0,개인_0,개인_0,개인_0,2025-03-28-20
1,서울형 키즈카페 시립 뚝섬자벌레점,https://icare.seoul.go.kr/icare/user/kidsCafeR...,공용_1,공용_0,공용_0,공용_0,공용_0,공용_1,공용_0,공용_0,공용_0,공용_14,2025-03-28-20
2,서울형 키즈카페 시립 목동점,https://icare.seoul.go.kr/icare/user/kidsCafeR...,개인_0,개인_0,개인_1,개인_0,개인_0,NaN,NaN,NaN,NaN,NaN,2025-03-28-20
3,서울형 키즈카페 강남구 역삼1동점,https://icare.seoul.go.kr/icare/user/kidsCafeR...,개인_0,개인_0,개인_0,NaN,NaN,개인_0,개인_0,개인_0,NaN,NaN,2025-03-28-20
4,서울형 키즈카페 강동구 고덕2동점(아이·맘 강동),https://icare.seoul.go.kr/icare/user/kidsCafeR...,개인_0,개인_0,개인_0,NaN,NaN,개인_0,개인_0,개인_0,NaN,NaN,2025-03-28-20


In [18]:
df2.head()

,키즈카페 이름,자치구,주소,이용연령_min,이용연령_max,이용정원_개인,이용정원_단체,예약 신청 URL
0,서울형 키즈카페 시립 1호점,동작구,서울특별시 동작구 노량진로 10 서울가족플라자 지하2층 (대방동),4,10,33,33,https://icare.seoul.go.kr/icare/user/kidsCafeR...
1,서울형 키즈카페 시립 뚝섬자벌레점,광진구,서울특별시 광진구 강변북로 2202 2층 꿈틀나루 (자양동),0,6,43,43,https://icare.seoul.go.kr/icare/user/kidsCafeR...
2,서울형 키즈카페 시립 목동점,양천구,서울특별시 양천구 안양천로 1131 지식산업센터 2층 (목동),3,12,24,24,https://icare.seoul.go.kr/icare/user/kidsCafeR...
3,서울형 키즈카페 강남구 역삼1동점,강남구,서울특별시 강남구 언주로107길 4 2층 (역삼동) 2층 서울형 키즈카페,0,6,24,24,https://icare.seoul.go.kr/icare/user/kidsCafeR...
4,서울형 키즈카페 강동구 고덕2동점(아이·맘 강동),강동구,서울특별시 강동구 고덕로 353 일반상가 2층 (농폅건물2층),0,6,12,12,https://icare.seoul.go.kr/icare/user/kidsCafeR...


In [19]:
# 정원값 삽입
df1['개인 정원']=df2['이용정원_개인']
df1['단체 정원']=df2['이용정원_단체']
df1.head()

,키즈카페 이름,예약 신청 URL,0329_1회,0329_2회,0329_3회,0329_4회,0329_5회,0330_1회,0330_2회,0330_3회,0330_4회,0330_5회,crawling_dt,개인 정원,단체 정원
0,서울형 키즈카페 시립 1호점,https://icare.seoul.go.kr/icare/user/kidsCafeR...,개인_0,개인_0,개인_0,개인_0,개인_0,개인_0,개인_0,개인_0,개인_0,개인_0,2025-03-28-20,33,33
1,서울형 키즈카페 시립 뚝섬자벌레점,https://icare.seoul.go.kr/icare/user/kidsCafeR...,공용_1,공용_0,공용_0,공용_0,공용_0,공용_1,공용_0,공용_0,공용_0,공용_14,2025-03-28-20,43,43
2,서울형 키즈카페 시립 목동점,https://icare.seoul.go.kr/icare/user/kidsCafeR...,개인_0,개인_0,개인_1,개인_0,개인_0,NaN,NaN,NaN,NaN,NaN,2025-03-28-20,24,24
3,서울형 키즈카페 강남구 역삼1동점,https://icare.seoul.go.kr/icare/user/kidsCafeR...,개인_0,개인_0,개인_0,NaN,NaN,개인_0,개인_0,개인_0,NaN,NaN,2025-03-28-20,24,24
4,서울형 키즈카페 강동구 고덕2동점(아이·맘 강동),https://icare.seoul.go.kr/icare/user/kidsCafeR...,개인_0,개인_0,개인_0,NaN,NaN,개인_0,개인_0,개인_0,NaN,NaN,2025-03-28-20,12,12


## 6. 예약정원 및 이용률 계산.

In [20]:
# 1. crawling_dt 위치 기준으로 예약 회차 컬럼 추출
crawling_dt_idx = df1.columns.get_loc('crawling_dt')
target_cols = df1.columns[2:crawling_dt_idx]

# 2. 개인/공용/단체 회차 수 카운트
def count_reservations(row):
    count_personal = 0
    count_group = 0
    for col in target_cols:
        val = row[col]
        if pd.isna(val): continue
        if isinstance(val, str):
            if val.startswith('공용_') or val.startswith('개인_'):
                count_personal += 1
            elif val.startswith('단체_'):
                count_group += 1
    return pd.Series([count_personal, count_group])

df1[['count_personal', 'count_group']] = df1.apply(count_reservations, axis=1)


In [21]:
# 3. 예약 텍스트를 실제 숫자로 변환
def transform_cell(value, personal_limit, group_limit):
    if pd.isna(value):
        return np.nan
    if isinstance(value, str):
        if value.startswith('공용_') or value.startswith('개인_'):
            used = int(value.split('_')[1])
            return personal_limit - used
        elif value.startswith('단체_') :
            used = int(value.split('_')[1])
            return group_limit - used
    return value

for col in target_cols:
    df1[col] = df1.apply(lambda row: transform_cell(row[col], row['개인 정원'], row['단체 정원']), axis=1)

In [22]:
# 4. 정원 합 계산
df1['정원 합'] = df1['count_personal'] * df1['개인 정원'] + df1['count_group'] * df1['단체 정원']



In [23]:
# 5. 회차값 숫자형으로 변환 (혹시 남아있는 문자열 예외 처리)
df1[target_cols] = df1[target_cols].apply(pd.to_numeric, errors='coerce')



In [24]:
# 6. 최종 합 계산 (모든 회차의 남은 좌석 합)
df1['최종 합'] = df1[target_cols].sum(axis=1)



In [25]:
# 7. 이용률 계산 (비율 * 100 → 백분율)
df1['이용률'] = df1.apply(
    lambda row: (row['최종 합'] / row['정원 합']) * 100 if row['정원 합'] != 0 else np.nan,
    axis=1
)

In [26]:
crawling_dt_idx = df1.columns.get_loc('crawling_dt')
target_cols = df1.columns[crawling_dt_idx+1:]
df1 = df1[['키즈카페 이름', '예약 신청 URL', '개인 정원', '단체 정원'] + list(target_cols)]
df1

,키즈카페 이름,예약 신청 URL,개인 정원,단체 정원,개인 정원,단체 정원,count_personal,count_group,정원 합,최종 합,이용률
0,서울형 키즈카페 시립 1호점,https://icare.seoul.go.kr/icare/user/kidsCafeR...,33,33,33,33,10,0,330,330.0,100.000000
1,서울형 키즈카페 시립 뚝섬자벌레점,https://icare.seoul.go.kr/icare/user/kidsCafeR...,43,43,43,43,10,0,430,414.0,96.279070
2,서울형 키즈카페 시립 목동점,https://icare.seoul.go.kr/icare/user/kidsCafeR...,24,24,24,24,5,0,120,119.0,99.166667
3,서울형 키즈카페 강남구 역삼1동점,https://icare.seoul.go.kr/icare/user/kidsCafeR...,24,24,24,24,6,0,144,144.0,100.000000
4,서울형 키즈카페 강동구 고덕2동점(아이·맘 강동),https://icare.seoul.go.kr/icare/user/kidsCafeR...,12,12,12,12,6,0,72,72.0,100.000000
...,...,...,...,...,...,...,...,...,...,...,...
70,은평아이맘놀이터 수색동점,https://icare.seoul.go.kr/icare/user/kidsCafeR...,14,14,14,14,0,0,0,0.0,NaN
71,서울형 키즈카페 종로구 혜화동점(종로 혜명 아이들 상상놀이터),https://icare.seoul.go.kr/icare/user/kidsCafeR...,25,30,25,30,1,0,25,7.0,28.000000
72,서울형 키즈카페 중구 중림동점(노리몽땅),https://icare.seoul.go.kr/icare/user/kidsCafeR...,38,38,38,38,8,0,304,75.0,24.671053
73,서울형 키즈카페 중랑구 망우본동점(중랑실내놀이터 양원),https://icare.seoul.go.kr/icare/user/kidsCafeR...,73,80,73,80,8,0,584,408.0,69.863014


## 7. 기존 통합 데이터와 병합 및 평일/주말 예약 합산 및 이용률 계산
총합 데이터를 불러와 현재 df1 데이터를 반영합니다.

In [27]:
# total 데이터 불러오기
df4=pd.read_csv('/Users/jihunlee/Library/Mobile Documents/com~apple~CloudDocs/Study/Data:AI/공모전/이용자 데이터/total_visitor.csv')

In [28]:
df4

,키즈카페 이름,평일 정원합,평일 최종합,주말 정원합,주말 최종합,평일 이용률,주말 이용률
0,서울형 키즈카페 시립 1호점,858,298.0,990.0,988.0,34.731935,99.797980
1,서울형 키즈카페 시립 뚝섬자벌레점,1118,381.0,1290.0,1175.0,34.078712,91.085271
2,서울형 키즈카페 시립 목동점,480,164.0,360.0,359.0,34.166667,99.722222
3,서울형 키즈카페 강남구 역삼1동점,576,163.0,432.0,382.0,28.298611,88.425926
4,서울형 키즈카페 강동구 고덕2동점(아이·맘 강동),264,165.0,216.0,216.0,62.500000,100.000000
...,...,...,...,...,...,...,...
72,서울형 키즈카페 중구 중림동점(노리몽땅),950,184.0,912.0,256.0,19.368421,28.070175
73,서울형 키즈카페 중랑구 망우본동점(중랑실내놀이터 양원),1825,421.0,1752.0,1214.0,23.068493,69.292237
74,서울형 키즈카페 중랑구 면목4동점(중랑 실내놀이터),965,215.0,805.0,532.0,22.279793,66.086957
75,서울형키즈카페 시립 옴팡점,900,159.0,600.0,592.0,17.666667,98.666667


In [29]:
# 평일일 경우
df4['평일 정원합']=df4['평일 정원합']+df1['정원 합']
df4['평일 최종합']=df4['평일 최종합']+df1['최종 합']


In [ ]:
# 주말일 경우
df4['주말 정원합']=df4['주말 정원합']+df1['정원 합']
df4['주말 최종합']=df4['주말 최종합']+df1['최종 합']


In [30]:
# 비율 구하기
df4['평일 이용률']=df4['평일 최종합']*100/df4['평일 정원합']
df4['주말 이용률']=df4['주말 최종합']*100/df4['주말 정원합']

In [31]:
df4

,키즈카페 이름,평일 정원합,평일 최종합,주말 정원합,주말 최종합,평일 이용률,주말 이용률
0,서울형 키즈카페 시립 1호점,1188.0,628.0,990.0,988.0,52.861953,99.797980
1,서울형 키즈카페 시립 뚝섬자벌레점,1548.0,795.0,1290.0,1175.0,51.356589,91.085271
2,서울형 키즈카페 시립 목동점,600.0,283.0,360.0,359.0,47.166667,99.722222
3,서울형 키즈카페 강남구 역삼1동점,720.0,307.0,432.0,382.0,42.638889,88.425926
4,서울형 키즈카페 강동구 고덕2동점(아이·맘 강동),336.0,237.0,216.0,216.0,70.535714,100.000000
...,...,...,...,...,...,...,...
72,서울형 키즈카페 중구 중림동점(노리몽땅),1254.0,259.0,912.0,256.0,20.653907,28.070175
73,서울형 키즈카페 중랑구 망우본동점(중랑실내놀이터 양원),2409.0,829.0,1752.0,1214.0,34.412619,69.292237
74,서울형 키즈카페 중랑구 면목4동점(중랑 실내놀이터),1210.0,381.0,805.0,532.0,31.487603,66.086957
75,서울형키즈카페 시립 옴팡점,NaN,NaN,600.0,592.0,NaN,98.666667


## 8. 주말 이용률 기준 이진 분류
주말 이용률이 80 이상이면 1, 아니면 0으로 표시합니다.

In [32]:
df4['result'] = (df4['주말 이용률'] >= 80).astype(int)

In [33]:
df4

,키즈카페 이름,평일 정원합,평일 최종합,주말 정원합,주말 최종합,평일 이용률,주말 이용률,result
0,서울형 키즈카페 시립 1호점,1188.0,628.0,990.0,988.0,52.861953,99.797980,1
1,서울형 키즈카페 시립 뚝섬자벌레점,1548.0,795.0,1290.0,1175.0,51.356589,91.085271,1
2,서울형 키즈카페 시립 목동점,600.0,283.0,360.0,359.0,47.166667,99.722222,1
3,서울형 키즈카페 강남구 역삼1동점,720.0,307.0,432.0,382.0,42.638889,88.425926,1
4,서울형 키즈카페 강동구 고덕2동점(아이·맘 강동),336.0,237.0,216.0,216.0,70.535714,100.000000,1
...,...,...,...,...,...,...,...,...
72,서울형 키즈카페 중구 중림동점(노리몽땅),1254.0,259.0,912.0,256.0,20.653907,28.070175,0
73,서울형 키즈카페 중랑구 망우본동점(중랑실내놀이터 양원),2409.0,829.0,1752.0,1214.0,34.412619,69.292237,0
74,서울형 키즈카페 중랑구 면목4동점(중랑 실내놀이터),1210.0,381.0,805.0,532.0,31.487603,66.086957,0
75,서울형키즈카페 시립 옴팡점,NaN,NaN,600.0,592.0,NaN,98.666667,1
